In [8]:
import pyquist as pq
import numpy as np

fs = 44100
t = np.arange(44100) / fs
audio = pq.Audio.from_array(np.sin(2 * np.pi * 880 * t), fs)
print(audio.shape, audio.peak_gain)
pq.play(audio)

(44100, 1) 0.9999997615814209


In [11]:
from pyquist.notebook import play

url = "https://github.com/librosa/data/raw/refs/heads/main/audio/198-209-0000.hq.ogg"

audio = pq.Audio.from_url(url)
audio.normalize(peak_dbfs=-18.0)
print(audio.peak_gain)
play(audio, safe=False, normalize=True)
play(audio, safe=False, normalize=False)

0.1258925348520279
